In [13]:
#PSYCOPG2

#This jupyter notebook demonstrates the usage of the Psycopg2 and Pandas library in Python.
#The following SQL/Python skills are demonstrated:

#1 Basic interaction between database and python (pandas dataframe conversion)
#2 Building a basic data pipeline using Psycopg2
#3 Data Analysis using the Pandas library 

    # SQL SPECIFIC 

    #1 Basic SELECT SQL statement
    #2 Aggregate Functions AVG, SUM, COUNT
    #3 Basic GROUP BY SQL statement
    #4 CREATE SQL Statement
    #5 INSERT SQL Statement

In [14]:
#Most basic SQL statement to demonstrate connection between Python and SQL database
import psycopg2 as pg2
import pandas as pd

conn = pg2.connect(database='dvdrental',user='postgres',password='Packers1')
cur = conn.cursor()
cur.execute("SELECT * FROM film;")
data = pd.DataFrame(cur.fetchall())
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,133,Chamber Italian,A Fateful Reflection of a Moose And a Husband ...,2006,1,7,4.99,117,14.99,NC-17,2013-05-26 14:50:58.951,[Trailers],'chamber':1 'fate':4 'husband':11 'italian':2 ...
1,384,Grosse Wonderful,A Epic Drama of a Cat And a Explorer who must ...,2006,1,5,4.99,49,19.99,R,2013-05-26 14:50:58.951,[Behind the Scenes],'australia':18 'cat':8 'drama':5 'epic':4 'exp...
2,8,Airport Pollock,A Epic Tale of a Moose And a Girl who must Con...,2006,1,6,4.99,54,15.99,R,2013-05-26 14:50:58.951,[Trailers],'airport':1 'ancient':18 'confront':14 'epic':...
3,98,Bright Encounters,A Fateful Yarn of a Lumberjack And a Feminist ...,2006,1,4,4.99,73,12.99,PG-13,2013-05-26 14:50:58.951,[Trailers],'boat':20 'bright':1 'conquer':14 'encount':2 ...
4,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'academi':1 'battl':15 'canadian':20 'dinosaur...


In [15]:
#Basic GROUP BY SQL statement
cur.execute("SELECT rating, ROUND(AVG(rental_rate),2), ROUND(AVG(rental_duration),2), ROUND(AVG(replacement_cost),2) FROM film GROUP BY rating")
data_1 = pd.DataFrame(cur.fetchall())
data_1.head(5)

,0,1,2,3
0,R,2.94,4.77,20.23
1,PG,3.05,5.08,18.96
2,NC-17,2.97,5.14,20.14
3,G,2.89,4.84,20.12
4,PG-13,3.03,5.05,20.40


In [16]:
#Creating new Pandas Columns/Dataframes based on data pulled from database
cur.execute("SELECT rental_date, return_date FROM rental ORDER BY rental_date ASC")
data_2 = pd.DataFrame(cur.fetchall())
data_2["timedelta"] = data_2[1] - data_2[0]
data_2.head(5)

,0,1,timedelta
0,2005-05-24 22:53:30,2005-05-26 22:04:30,1 days 23:11:00
1,2005-05-24 22:54:33,2005-05-28 19:40:33,3 days 20:46:00
2,2005-05-24 23:03:39,2005-06-01 22:12:39,7 days 23:09:00
3,2005-05-24 23:04:41,2005-06-03 01:43:41,9 days 02:39:00
4,2005-05-24 23:05:21,2005-06-02 04:33:21,8 days 05:28:00


In [17]:
#Creating a table in the database, Inserting data into the newly created table
#Calling the data to prove the records where uploaded into the Postgres database

cur.execute('DROP TABLE IF EXISTS employee1')
create_tbl = '''CREATE TABLE IF NOT EXISTS employee1 (
    id int PRIMARY KEY,
    name varchar(40) NOT NULL,
    salary int,
    dept_id varchar(30)) '''

cur.execute(create_tbl)

insert_script = 'INSERT INTO employee1 (id, name, salary, dept_id) VALUES (%s, %s, %s, %s)'
insert_value = [(1, 'James Bond', 12000, 'D1'),
                (2, 'McLovin', 100000, 'D2'),
                (3, 'Stephen King', 5000000, 'D3')]
for record in insert_value:
    cur.execute(insert_script, record)

conn.commit()
cur.execute("SELECT * FROM employee1")
data_3 = pd.DataFrame(cur.fetchall())
data_3

,0,1,2,3
0,1,James Bond,12000,D1
1,2,McLovin,100000,D2
2,3,Stephen King,5000000,D3


In [33]:
#Loading a CSV file into a postgres database using Psycopg2
#Calling the data to prove the CSV file was uploaded into the Postgres database
import numpy as np 
import os
import datetime as dt

sales = pd.read_csv('Sales Data.csv')
file = 'Sales Data'
sales['date'] = pd.to_datetime(sales['date'])
clean_table = file.lower().replace(" ","").replace("/","_")
sales.columns = [x.lower().replace(" ","").replace("/","_") for x in sales.columns]

replacements = {'object':'varchar',
                'float64':'float',
                'int64':'int',
                'datetime64[ns]':'timestamp',
                'timedelta64[ns]':'varchar'}

col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(sales.columns,sales.dtypes.replace(replacements)))

#Open a database connection
conn = pg2.connect(database='dvdrental',user='postgres',password='Packers1')
cur = conn.cursor()
print('Opened database succesfully')

#Drop tables with same name
cur.execute("DROP TABLE IF EXISTS salesdata;")

#Create table
create_tbl = '''CREATE TABLE salesdata (
                id int, 
                date timestamp, 
                state varchar, 
                product varchar, 
                qty int, 
                unitprice float)'''

cur.execute(create_tbl)

#Insert Values to table
#Save sales dataframe to csv
sales.to_csv('salesdata.csv', header=sales.columns, index=False, encoding='utf-8')
my_file = open('salesdata.csv')

#Upload to Postgres database
SQL_STATEMENT = """
COPY salesdata FROM STDIN WITH 
    CSV
    HEADER
    DELIMITER AS ',' 
"""
cur.copy_expert(sql=SQL_STATEMENT, file=my_file)
cur.execute("GRANT SELECT ON TABLE salesdata TO PUBLIC")
conn.commit()

cur.execute("SELECT * FROM salesdata")
data = pd.DataFrame(cur.fetchall())
cur.close()
data

Opened database succesfully


,0,1,2,3,4,5
0,1,2022-01-01,Alabama,MacBook Pro,3,1499.99
1,2,2022-01-02,Georgia,iMac,1,1799.99
2,3,2022-01-03,Florida,Keyboard,5,99.99
3,4,2022-01-04,South Carolina,Surface,1,999.99
4,5,2022-01-05,Texas,Headphones,3,249.99
5,6,2022-01-06,Mississipi,MacBook Pro,5,1299.99
6,7,2022-01-07,Louisiana,iMac,4,2249.99
7,8,2022-01-08,Kentucky,Keyboard,4,124.99
8,9,2022-01-09,Tennessee,Surface,2,1356.87
9,10,2022-01-10,Virginia,Headphones,5,199.99
